In [1]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 47.3 MB/s  0:00:00
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 55.9 MB/s  0:00:00
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)

   ----- ----------------------------------  3/21 [tenacity]
   ------------- --------------------------  7/21 [greenlet]
   ------------- --------------------------  7/21 [greenlet]
   ----------------- ----------------------  9/21 [SQLAlchemy]
   ----------------- ----------------------  9/21 [SQLAlchemy]
   --------------

In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [5]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [6]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1755668385, 'localtime': '2025-08-19 22:39'}, 'current': {'last_updated_epoch': 1755667800, 'last_updated': '2025-08-19 22:30', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 7.6, 'wind_kph': 12.2, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 84, 'cloud': 50, 'feelslike_c': 16.7, 'feelslike_f': 62.1, 'windchill_c': 13.0, 'windchill_f': 55.5, 'heatindex_c': 13.8, 'heatindex_f': 56.8, 'dewpoint_c': 13.2,

In [7]:
pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [8]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini:  ········


In [9]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

In [10]:
model_with_tools = model.bind_tools(tools)

In [11]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [12]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in SF'}, 'id': 'ffff72c6-ab5e-4b7d-a466-a848492638ea', 'type': 'tool_call'}]


In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [15]:
input_message = {"role": "user", "content": "Search for the weather in Chennai"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Chennai
================================== Ai Message ==================================
Tool Calls:
  tavily_search (5b62eaee-35f4-44a6-95ef-82d0dd57a140)
 Call ID: 5b62eaee-35f4-44a6-95ef-82d0dd57a140
  Args:
    query: weather in Chennai
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Chennai", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Chennai", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.0833, 'lon': 80.2833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1755668592, 'localtime': '2025-08-20 11:13'}, 'current': {'last_updated_epoch': 1755667800, 'last_updated': '2025-08-20 11:00', 'temp_c': 34.2, 'temp_f': 93.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy',

In [16]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in Chennai
================================== Ai Message ==================================
Tool Calls:
  tavily_search (9838a1d9-33f3-41f9-8e1d-a870c747939e)
 Call ID: 9838a1d9-33f3-41f9-8e1d-a870c747939e
  Args:
    query: weather in Chennai
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Chennai", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Chennai", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.0833, 'lon': 80.2833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1755668592, 'localtime': '2025-08-20 11:13'}, 'current': {'last_updated_epoch': 1755667800, 'last_updated': '2025-08-20 11:00', 'temp_c': 34.2, 'temp_f': 93.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy',

In [17]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in Chennai is partly cloudy with a temperature of 34.2°C (93.6°F). The wind is blowing from the West at 26.3 km/h (16.3 mph).| The humidity is 53%, and it feels like 39.2°C (102.6°F). The UV index is 11.7. The maximum temperature expected today is 34°C and the| minimum is 27°C.|

In [22]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [23]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [24]:
input_message = {"role": "user", "content": "Hi, I'm Shravan!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Shravan!
================================== Ai Message ==================================

Hi Shravan! How can I help you today?


In [26]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

As I mentioned, I don't have memory of past conversations. You just told me your name is Shravan.
